In [1]:
import cv2
import numpy as np 
import glob
from tqdm import tqdm
import PIL.ExifTags
import PIL.Image
from matplotlib import pyplot as plt 

In [2]:
#=====================================
# Function declarations
#=====================================

#Function to create point cloud file
def create_output(vertices, colors, filename):
    colors = colors.reshape(-1,3)
    vertices = np.hstack([vertices.reshape(-1,3),colors])

    ply_header = '''ply
        format ascii 1.0
        element vertex %(vert_num)d
        property float x
        property float y
        property float z
        property uchar red
        property uchar green
        property uchar blue
        end_header
        '''
    with open(filename, 'w') as f:
        f.write(ply_header %dict(vert_num=len(vertices)))
        np.savetxt(f,vertices,'%f %f %f %d %d %d')

#Function that Downsamples image x number (reduce_factor) of times. 
def downsample_image(image, reduce_factor):
    for i in range(0,reduce_factor):
        #Check if image is color or grayscale
        if len(image.shape) > 2:
            row,col = image.shape[:2]
        else:
            row,col = image.shape

        image = cv2.pyrDown(image, dstsize= (col//2, row // 2))
    return image


In [3]:


#=========================================================
# Stereo 3D reconstruction 
#=========================================================

#Load camera parameters
ret = np.load('./calibration/camera_params/ret.npy')
K = np.load('./calibration/camera_params/K.npy')
dist = np.load('./calibration/camera_params/dist.npy')

#Specify image paths
img_path1 = 'Left.jpg'
img_path2 = 'Right.jpg'

#Load pictures
img_1 = cv2.imread(img_path1)
img_2 = cv2.imread(img_path2)

#Get height and width. Note: It assumes that both pictures are the same size. They HAVE to be same size and height. 
h,w = img_2.shape[:2]

#Get optimal camera matrix for better undistortion 
new_camera_matrix, roi = cv2.getOptimalNewCameraMatrix(K,dist,(w,h),1,(w,h))

#Undistort images
img_1_undistorted = cv2.undistort(img_1, K, dist, None, new_camera_matrix)
img_2_undistorted = cv2.undistort(img_2, K, dist, None, new_camera_matrix)

#Downsample each image 3 times (because they're too big)
img_1_downsampled = downsample_image(img_1_undistorted,3)
img_2_downsampled = downsample_image(img_2_undistorted,3)

#cv2.imwrite('undistorted_left.jpg', img_1_downsampled)
#cv2.imwrite('undistorted_right.jpg', img_2_downsampled)


#Set disparity parameters
#Note: disparity range is tuned according to specific parameters obtained through trial and error. 
win_size = 5
min_disp = -1
max_disp = 63 #min_disp * 9
num_disp = max_disp - min_disp # Needs to be divisible by 16

#Create Block matching object. 
stereo = cv2.StereoSGBM_create(minDisparity= min_disp,
    numDisparities = num_disp,
    blockSize = 5,
    uniquenessRatio = 5,
    speckleWindowSize = 5,
    speckleRange = 5,
    disp12MaxDiff = 2,
    P1 = 8*3*win_size**2,#8*3*win_size**2,
    P2 =32*3*win_size**2) #32*3*win_size**2)

#Compute disparity map
print ("\nComputing the disparity  map...")
disparity_map = stereo.compute(img_1_downsampled, img_2_downsampled)

#Show disparity map before generating 3D cloud to verify that point cloud will be usable. 
plt.imshow(disparity_map,'gray')
plt.show()

#Generate  point cloud. 
print ("\nGenerating the 3D map...")

#Get new downsampled width and height 
h,w = img_2_downsampled.shape[:2]

#Load focal length. 
focal_length = np.load('./calibration/camera_params/FocalLength.npy')

#Perspective transformation matrix
#This transformation matrix is from the openCV documentation, didn't seem to work for me. 
Q = np.float32([[1,0,0,-w/2.0],
                [0,-1,0,h/2.0],
                [0,0,0,-focal_length],
                [0,0,1,0]])

#This transformation matrix is derived from Prof. Didier Stricker's power point presentation on computer vision. 
#Link : https://ags.cs.uni-kl.de/fileadmin/inf_ags/3dcv-ws14-15/3DCV_lec01_camera.pdf
Q2 = np.float32([[1,0,0,0],
                [0,-1,0,0],
                [0,0,focal_length*0.05,0], #Focal length multiplication obtained experimentally. 
                [0,0,0,1]])

#Reproject points into 3D
points_3D = cv2.reprojectImageTo3D(disparity_map, Q2)
#Get color points
colors = cv2.cvtColor(img_1_downsampled, cv2.COLOR_BGR2RGB)

#Get rid of points with value 0 (i.e no depth)
mask_map = disparity_map > disparity_map.min()

#Mask colors and points. 
output_points = points_3D[mask_map]
output_colors = colors[mask_map]

#Define name for output file
output_file = 'reconstructed.ply'

#Generate point cloud 
print ("\n Creating the output file... \n")
create_output(output_points, output_colors, output_file)

FileNotFoundError: [Errno 2] No such file or directory: './calibration/camera_params/ret.npy'

In [1]:
import cv2
import numpy as np 
import glob
from tqdm import tqdm
import PIL.ExifTags
import PIL.Image
from matplotlib import pyplot as plt 

In [2]:
#Specify image paths
img_path1 = 'Left.jpg'
img_path2 = 'Right.jpg'

#Load pictures
img_1 = cv2.imread(img_path1)
img_2 = cv2.imread(img_path2)
 
# Convert image to RGB
img_1 = cv2.cvtColor(img_1, cv2.COLOR_BGR2RGB)
img_2 = cv2.cvtColor(img_2, cv2.COLOR_BGR2RGB)

# Convert image to gray scale
img_1_gray = cv2.cvtColor(img_1, cv2.COLOR_RGB2GRAY)
img_2_gray = cv2.cvtColor(img_2, cv2.COLOR_RGB2GRAY)

In [3]:
orb = cv2.ORB_create()

keypoints1, descriptor1 = orb.detectAndCompute(img_1_gray, None)
keypoints2, descriptor2 = orb.detectAndCompute(img_2_gray, None)

In [4]:
# Create a Brute Force Matcher object.
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck = True)

# Perform the matching between the ORB descriptors of the training image and the test image
matches = bf.match(descriptor1, descriptor2)

# The matches with shorter distance are the ones we want.
matches = sorted(matches, key = lambda x : x.distance)



In [46]:
list_kp1 = [keypoints1[mat.queryIdx].pt for mat in matches] 
list_kp2 = [keypoints2[mat.trainIdx].pt for mat in matches]
#projPoints1 = np.int32(list_kp1)
#projPoints2 = np.int32(list_kp2)
projPoints1 = cv2.UMat(np.array([list_kp1]))
projPoints2 = cv2.UMat(np.array([list_kp2]))
#projPoints1 = [cv2.UMat(x) for x in list_kp1]
#projPoints2 = [cv2.UMat(x) for x in list_kp2]

In [47]:
# Load camera parameters
# get projection matrix in the new (rectified) coordinate systems for the first camera
# and projection matrix in the new (rectified) coordinate systems for the second camera.
line = './calibration/camera_params/stereo_params/'
ret = np.load(line + 'ret.npy')
K_left = np.load(line + 'K_left.npy')
K_right = np.load(line + 'K_right.npy')
dist_left = np.load(line + 'dist_left.npy')
dist_right = np.load(line + 'dist_right.npy')
R = np.load(line + 'R.npy')
T = np.load(line + 'T.npy')
image_size = np.load(line + 'image_size.npy')
image_size = (image_size[0], image_size[1])
R1, R2, P1, P2, Q, roi1, roi2 = cv2.stereoRectify(cameraMatrix1 = K_left, 
                                                  cameraMatrix2 = K_right, 
                                                  distCoeffs1 = dist_left, 
                                                  distCoeffs2 = dist_right, 
                                                  imageSize = image_size, 
                                                  R = R, 
                                                  T = T,
                                                  alpha=1)


In [48]:
points4D = cv2.triangulatePoints(P1, P2, projPoints1, projPoints2) 

In [58]:
points4D.get()[:3].T

array([[-7.59659226e-10,  2.64882112e-10,  7.98688526e-11],
       [-7.77269126e-10,  2.70827793e-10,  8.17357582e-11],
       [-7.48640067e-10,  2.61122860e-10,  7.87682900e-11],
       [-5.29220350e-10,  1.66652722e-10,  5.03054350e-11],
       [-7.44242841e-10,  2.59570021e-10,  7.82907863e-11],
       [-5.38416109e-10,  1.69525580e-10,  5.11796705e-11],
       [-7.84910594e-10,  2.33607300e-10,  7.97899169e-11],
       [-7.30701486e-10,  2.18597324e-10,  7.44262826e-11],
       [-8.96686191e-10,  3.05923817e-10,  9.21652717e-11],
       [-8.80066248e-10,  3.00210105e-10,  9.25555946e-11],
       [-7.29541364e-10,  2.18280758e-10,  7.43151816e-11],
       [-8.60546395e-10,  2.93701857e-10,  8.84505839e-11],
       [-9.16715212e-10,  3.12922589e-10,  9.64121597e-11],
       [ 1.52604563e-11, -5.18936619e-12, -1.56878174e-12],
       [-7.88851682e-10,  2.75017653e-10,  8.29427621e-11],
       [-7.42794453e-10,  2.58997611e-10,  7.81396197e-11],
       [-7.46994067e-10,  2.60465669e-10

In [40]:
projPoints1

<UMat 0x7f0986e1ca70>

In [41]:
list_kp1[0]

(903.6000366210938, 948.0000610351562)

In [42]:
np.array([list_kp1[0], list_kp1[1]])

array([[903.60003662, 948.00006104],
       [901.20001221, 951.60003662]])

In [43]:
np.array([list_kp1])

array([[[ 903.60003662,  948.00006104],
        [ 901.20001221,  951.60003662],
        [ 895.79541016,  949.54315186],
        [2009.        ,  961.        ],
        [ 897.60003662,  949.20001221],
        [2008.80004883,  961.20001221],
        [1259.7121582 , 1375.48815918],
        [1238.40002441, 1365.11999512],
        [1140.48010254,  946.9440918 ],
        [ 898.56005859, 1029.88806152],
        [1237.24816895, 1365.12011719],
        [1140.48010254,  945.56170654],
        [ 898.28375244, 1027.6763916 ],
        [1137.60009766,  946.80004883],
        [ 902.88006592,  948.96002197],
        [ 897.12005615,  950.40002441],
        [ 896.83209229,  950.40008545],
        [2009.31872559,  962.1505127 ],
        [2002.99865723, 1021.20684814],
        [ 897.86889648,  949.70892334],
        [1232.06408691, 1437.69616699],
        [1604.        , 1897.        ],
        [2010.2401123 , 1025.2800293 ],
        [ 897.12005615, 1031.04003906],
        [1260.74902344, 1393.45935059],


In [44]:
np.array(list_kp1)

array([[ 903.60003662,  948.00006104],
       [ 901.20001221,  951.60003662],
       [ 895.79541016,  949.54315186],
       [2009.        ,  961.        ],
       [ 897.60003662,  949.20001221],
       [2008.80004883,  961.20001221],
       [1259.7121582 , 1375.48815918],
       [1238.40002441, 1365.11999512],
       [1140.48010254,  946.9440918 ],
       [ 898.56005859, 1029.88806152],
       [1237.24816895, 1365.12011719],
       [1140.48010254,  945.56170654],
       [ 898.28375244, 1027.6763916 ],
       [1137.60009766,  946.80004883],
       [ 902.88006592,  948.96002197],
       [ 897.12005615,  950.40002441],
       [ 896.83209229,  950.40008545],
       [2009.31872559,  962.1505127 ],
       [2002.99865723, 1021.20684814],
       [ 897.86889648,  949.70892334],
       [1232.06408691, 1437.69616699],
       [1604.        , 1897.        ],
       [2010.2401123 , 1025.2800293 ],
       [ 897.12005615, 1031.04003906],
       [1260.74902344, 1393.45935059],
       [1140.64624023,  9